PyVis is used to generate the network graphs
[PyVis](https://pyvis.readthedocs.io/en/latest/tutorial.html)

PlotLy Group Function is used to generate the distribution graphs
[Plotly](https://plotly.com/python/group-by/)


Some Acronyms:
IG: Interactive Graph
SP: Scatter Plot

In [3]:
from pyvis.network import Network
import pandas as pd
from pandas import DataFrame
import plotly.express as px
import textwrap

def extract_numbers(input_string):
    numbers_only = ''.join(char for char in input_string if char.isdigit())
    result_integer = int(numbers_only)
    return result_integer

def wrap_text(text, width=50):
    wrapper = textwrap.TextWrapper(width=width, break_long_words=False)
    return '\n'.join(wrapper.wrap(text))

def wrap_text_withBR(text, width=40):
    wrapper = textwrap.TextWrapper(width=width, break_long_words=False)
    return '<br>'.join(wrapper.wrap(text))

# def CenteredAlign(text, width=50):
#     wrapper = textwrap.TextWrapper(width=width, break_long_words=False)
#     return '<br>'.join(wrapper.wrap(text))
def processing(df):
  for column in df.columns:
    df[column] = df[column].str.lstrip().str.rstrip()
    return df

filename = "ReACT_Dataset.xlsx"

MainSheet = pd.read_excel(filename, sheet_name='main', skiprows=0)
FeatureDescriptionSheet = pd.read_excel(filename, sheet_name='FeatureDescription', skiprows=0)
EntityDescriptionSheet = pd.read_excel(filename, sheet_name='EntityDescription', skiprows=0)

# MainSheet = MainSheet.drop(columns=['Article Title', 'Link', 'SL'])

MainSheet = processing(MainSheet)
FeatureDescriptionSheet = processing(FeatureDescriptionSheet)
# ClassDescriptionSheet = processing(ClassDescriptionSheet)
EntityDescriptionSheet = processing(EntityDescriptionSheet)
# Display the modified DataFrame
MainSheet['ReACT_title'] = MainSheet['ReACT_title'].apply(wrap_text)

ReAct_Dict = dict(zip(MainSheet['ReACT_number'], MainSheet['ReACT_title']))
FeatureDescription_Dict = dict(zip(FeatureDescriptionSheet['FeatureName'], FeatureDescriptionSheet['DescriptionOfTheFeature']))
# ClassDescription_Dict = dict(zip(ClassDescriptionSheet['Class'], ClassDescriptionSheet['ClassDescription']))
EntityDescription_Dict = dict(zip(EntityDescriptionSheet['Entity'], EntityDescriptionSheet['EntityDescription']))
ReActAndTitle = MainSheet[['ReACT_number', 'ReACT_title']]
MainSheet.head()

,ReACT_number,ReACT_title,Entity_responsible,Features,References,Articles
0,ReACT-1,Ensure a team of friendly and supportive\ndeve...,"Project Maintainers, Project Leaders",num_act_dev,No. of reference: 3\n,"1. G. Avelino, E. Constantinou, M. T. Valente ..."
1,ReACT-1,Ensure a team of friendly and supportive\ndeve...,Community Managers,"c_nodes, c_edges, c_long_tail, c_mean_degree, ...",No. of reference: 1,"1. Sholler D, Steinmacher I, Ford D, Averick M..."
2,ReACT-2,"Include the new members in the project, levera...","Project Maintainers, Project Leaders",num_act_dev,No. of reference: 1,"1. G. Avelino, E. Constantinou, M. T. Valente ..."
3,ReACT-3,Utilize a common programming language,"Project Maintainers, Project Leaders",num_act_dev,No. of reference: 3\n,"1. G. Avelino, E. Constantinou, M. T. Valente ..."
4,ReACT-3,Utilize a common programming language,"Project Maintainers, Project Leaders","num_act_dev, top_c_fract, num_commits, c_nodes...",No. of reference: 1,"1. Ferreira F, Silva LL, Valente MT. Turnover ..."


#Plotting ReAct and Entity - Network Diagram

In [5]:
ReActAndEntity = MainSheet[['ReACT_number', 'Entity_responsible']]

b = DataFrame(ReActAndEntity.Entity_responsible.str.split(',').tolist(), index=ReActAndEntity.ReACT_number).stack()
b = b.reset_index()[[0, 'ReACT_number']]
b.columns = ['Entity', 'ReACT_number'] # renaming var1
ReActAndEntity = b
ReActAndEntity = processing(ReActAndEntity) # performing lstrip rstrip after the separation
# ReActAndEntity.Entity.value_counts()

###Unique values in the Entity Column
(You neeed description for each item in the EntityDescription Sheet)
I wrote this to determine the unique values available in the Entity Column. So that, later, I can define each of them in the EntityDescription sheet

In [6]:
unique_values = set(ReActAndEntity.Entity)

# Convert the unique values back to a list if needed
unique_values_list = list(unique_values)

print(unique_values_list)
for i in range(0,len(unique_values_list)):
  print(unique_values_list[i].lstrip().rstrip())

['Project Leaders', 'Developers', 'Code Reviewers', 'Owners', 'Design Team', 'Collaboration Coordinators', 'Community Moderators', 'Contributors', 'Documentation Team', 'Mentors', 'Quality Assurance Team', 'Architects', 'Project Managers', 'Project Maintainers', 'Community Managers', 'Core Developers', 'Technical Leads', 'Sponsors']
Project Leaders
Developers
Code Reviewers
Owners
Design Team
Collaboration Coordinators
Community Moderators
Contributors
Documentation Team
Mentors
Quality Assurance Team
Architects
Project Managers
Project Maintainers
Community Managers
Core Developers
Technical Leads
Sponsors


In [7]:
ReActTitleEntity = pd.merge(ReActAndTitle, ReActAndEntity, on='ReACT_number', how='outer')
ReActTitleEntity['EntityDescription'] = None

ReActEntityDescription = ReActTitleEntity

for i in range(0,len(ReActEntityDescription)):
  ReActEntityDescription.EntityDescription[i] = EntityDescription_Dict[ReActEntityDescription.Entity[i].lstrip().rstrip()]
ReAct_Title_Entity_EntityDescription = ReActEntityDescription

#The following line is added to remove the exact duplicate rows based on ['ReACT_number', 'Entity']
ReAct_Title_Entity_EntityDescription = ReAct_Title_Entity_EntityDescription.drop_duplicates(subset=['ReACT_number', 'Entity'])
ReAct_Title_Entity_EntityDescription.reset_index(inplace=True, drop=True)
ReAct_Title_Entity_EntityDescription

/tmp/ipykernel_70801/1196293776.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  ReActEntityDescription.EntityDescription[i] = EntityDescription_Dict[ReActEntityDescription.Entity[i].lstrip().rstrip()]
/tmp/ipykernel_70801/1196293776.py:7

,ReACT_number,ReACT_title,Entity,EntityDescription
0,ReACT-1,Ensure a team of friendly and supportive\ndeve...,Project Maintainers,Project maintainers in open-source software de...
1,ReACT-1,Ensure a team of friendly and supportive\ndeve...,Project Leaders,Project leaders in open-source software develo...
2,ReACT-1,Ensure a team of friendly and supportive\ndeve...,Community Managers,Community managers in open source oversee posi...
3,ReACT-10,Offer job support to the newcomer.,Project Managers,"In open-source software development, project m..."
4,ReACT-10,Offer job support to the newcomer.,Owners,"In open-source software development, owners ty..."
...,...,...,...,...
188,ReACT-98,Make it easy for newcomers build the system\nl...,Technical Leads,Technical leads in open-source software develo...
189,ReACT-98,Make it easy for newcomers build the system\nl...,Collaboration Coordinators,Collaboration Coordinators in open-source soft...
190,ReACT-99,Keep the issue list clean and triaged: Read th...,Code Reviewers,Code reviewers in open-source software develop...
191,ReACT-99,Keep the issue list clean and triaged: Read th...,Project Maintainers,Project maintainers in open-source software de...


In [9]:
df = ReAct_Title_Entity_EntityDescription
ReACT_number = df['ReACT_number'].values

ReACT_Title = df['ReACT_title'].values

Entity = df['Entity'].values
df['EntityDescription'] = df['EntityDescription'].apply(wrap_text)
EntityDescription = df['EntityDescription'].values


net = Network(directed =True)
for i in range(0,len(ReACT_number)):
  # print(actionable[i])
  net.add_node(ReACT_number[i], label=ReACT_number[i], title=ReACT_Title[i])
  net.add_node(Entity[i], label=Entity[i], title=EntityDescription[i], color = "#98BF64",shape='box')
  net.add_edge(Entity[i], ReACT_number[i], color = "#32612D")

# net.show_buttons(filter_=['physics'])
net.show('IG_Entity.html')

/tmp/ipykernel_70801/586009387.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EntityDescription'] = df['EntityDescription'].apply(wrap_text)


In [12]:
unique_counts = ReAct_Title_Entity_EntityDescription['Entity'].value_counts().reset_index()

# Rename the columns for clarity
unique_counts.columns = ['Entity Responsible for Performing ReACTs', 'Count of ReACTs']

# Define an array of colors
darker_shades = ['darkred', 'lightblue', 'lightgreen', '#FFABAB', '#91a8a8',
                 '#95ab8a', '#00a86b', '#8E44AD', '#2E4053', '#D35400', '#a37b65',
                 '#6C5B7B', '#C0C999', '#7D8CC4', '#FFA07A', '#88D8B0',
    '#826c7F', '#D8D8D8', '#5F758E', '#E2B464', '#649E7D',
    '#CAAE77', '#7A9E9F', '#FAE5D3', '#BAC9FF', '#E47878',
    '#D2F2D2', '#7C83FD', '#F5A623', '#A77B7B', '#D8D8D8']


# Create an interactive bar plot using Plotly Express
fig = px.bar(unique_counts, x='Count of ReACTs', y='Entity Responsible for Performing ReACTs', color='Entity Responsible for Performing ReACTs', color_discrete_sequence=darker_shades, orientation='h')

# Save the plot as an HTML file
fig.write_html('BarPlot_Entity.html')
# fig.show()

In [15]:
import plotly.graph_objects as go
import plotly.io as pio

subject = ReAct_Title_Entity_EntityDescription['Entity']
score = ReAct_Title_Entity_EntityDescription['ReACT_number']

TextToShow = "<b>"+ReAct_Title_Entity_EntityDescription['Entity'] + " are Responsible for Performing </b>"
TextToShow += "<b>"+ReAct_Title_Entity_EntityDescription['ReACT_number'] +"</b>"+ "<br>"
TextToShow += "<b>"+ReAct_Title_Entity_EntityDescription['ReACT_number']+"</b>: "
TextToShow += ReAct_Title_Entity_EntityDescription['ReACT_title'].apply(wrap_text_withBR)
TextToShow += "<br><b>" +ReAct_Title_Entity_EntityDescription['Entity'] +": </b>"
TextToShow += ReAct_Title_Entity_EntityDescription['EntityDescription'].apply(wrap_text_withBR)


data = [dict(
  type = 'scatter',
  x = subject,
  y = score,
  mode = 'markers',
  text = TextToShow,  # Set the 'text' property to the Y-axis values
  transforms = [dict(
    type = 'groupby',
    groups = subject
  )],
   marker=dict(
        symbol='square',  # Set the shape of the marker (e.g., 'circle', 'square', 'diamond', etc.)
        size=10  # Set the size of the circles (adjust as needed)

    )
)]

layout = dict(xaxis=dict(title='Features'),
              yaxis=dict(title='ReACT Number'),
              height=1200,
              width=1100)

fig_dict = dict(data=data, layout=layout)
# pio.show(fig_dict, validate=False)
pio.write_html(fig_dict, file='SP_Entity.html', auto_open=True, validate=False)

#Plotting ReAct and Features - Network Diagram

In [16]:
MainSheet

,ReACT_number,ReACT_title,Entity_responsible,Features,References,Articles
0,ReACT-1,Ensure a team of friendly and supportive\ndeve...,"Project Maintainers, Project Leaders",num_act_dev,No. of reference: 3\n,"1. G. Avelino, E. Constantinou, M. T. Valente ..."
1,ReACT-1,Ensure a team of friendly and supportive\ndeve...,Community Managers,"c_nodes, c_edges, c_long_tail, c_mean_degree, ...",No. of reference: 1,"1. Sholler D, Steinmacher I, Ford D, Averick M..."
2,ReACT-2,"Include the new members in the project, levera...","Project Maintainers, Project Leaders",num_act_dev,No. of reference: 1,"1. G. Avelino, E. Constantinou, M. T. Valente ..."
3,ReACT-3,Utilize a common programming language,"Project Maintainers, Project Leaders",num_act_dev,No. of reference: 3\n,"1. G. Avelino, E. Constantinou, M. T. Valente ..."
4,ReACT-3,Utilize a common programming language,"Project Maintainers, Project Leaders","num_act_dev, top_c_fract, num_commits, c_nodes...",No. of reference: 1,"1. Ferreira F, Silva LL, Valente MT. Turnover ..."
...,...,...,...,...,...,...
111,ReACT-101,Dismiss or identify outdated information: if ...,"Documentation Team, Community Moderators, Coll...","c_nodes, c_edges, c_long_tail, c_mean_degree",No. of reference: 1,"1. Steinmacher I, Gerosa MA. Fostering Free/Li..."
112,ReACT-102,Create a live FAQ section. Create FAQ sections...,"Documentation Team, Collaboration Coordinators...","c_nodes, c_edges, c_long_tail, c_mean_degree",No. of reference: 1,"1. Steinmacher I, Gerosa MA. Fostering Free/Li..."
113,ReACT-103,Keep the community informed about decisions. W...,Developers,num_act_dev,No. of reference: 1,"1. Steinmacher I, Treude C, Gerosa MA. Let me ..."
114,ReACT-104,Be proactive - search archives and resources\n...,Developers,num_act_dev,No. of reference: 1,"1. Steinmacher I, Treude C, Gerosa MA. Let me ..."


In [17]:
ReActAndFeature = MainSheet[['ReACT_number', 'Features']]

b = DataFrame(ReActAndFeature.Features.str.split(',').tolist(), index=ReActAndFeature.ReACT_number).stack()
b = b.reset_index()[[0, 'ReACT_number']]
b.columns = ['Features', 'ReACT_number'] # renaming var1
ReActAndFeature  = b
ReActAndFeature = processing(ReActAndFeature)
ReActAndFeature

,Features,ReACT_number
0,num_act_dev,ReACT-1
1,c_nodes,ReACT-1
2,c_edges,ReACT-1
3,c_long_tail,ReACT-1
4,c_mean_degree,ReACT-1
...,...,...
355,c_long_tail,ReACT-102
356,c_mean_degree,ReACT-102
357,num_act_dev,ReACT-103
358,num_act_dev,ReACT-104


In [18]:
ReActTitleFeature = pd.merge(ReActAndTitle, ReActAndFeature, on='ReACT_number', how='outer')
ReActTitleFeature['FeatureDescription'] = None
ReActAndReference = MainSheet[['ReACT_number', 'References', 'Articles']]

ReActFeatureDescription = ReActTitleFeature


for i in range(0,len(ReActFeatureDescription)):
 ReActFeatureDescription.FeatureDescription[i] = FeatureDescription_Dict[ReActFeatureDescription.Features[i].lstrip().rstrip()]
ReAct_Title_Feature_FeatureDescription = ReActFeatureDescription
ReAct_Title_Feature_FeatureDescription_Reference = pd.merge(ReAct_Title_Feature_FeatureDescription, ReActAndReference, on='ReACT_number', how='outer')
ReAct_Title_Feature_FeatureDescription_Reference

#The following line is added to remove the exact duplicate rows based on ['ReACT_number', 'Features']
ReAct_Title_Feature_FeatureDescription_Reference = ReAct_Title_Feature_FeatureDescription_Reference.drop_duplicates(subset=['ReACT_number', 'Features'])
ReAct_Title_Feature_FeatureDescription_Reference.reset_index(inplace=True, drop=True)
ReAct_Title_Feature_FeatureDescription_Reference

/tmp/ipykernel_70801/1646877405.py:9: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


/tmp/ipykernel_70801/1646877405.py:9: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained ass

,ReACT_number,ReACT_title,Features,FeatureDescription,References,Articles
0,ReACT-1,Ensure a team of friendly and supportive\ndeve...,num_act_dev,The number of Active Developers is the count o...,No. of reference: 3\n,"1. G. Avelino, E. Constantinou, M. T. Valente ..."
1,ReACT-1,Ensure a team of friendly and supportive\ndeve...,c_nodes,The number of active nodes in the technical ne...,No. of reference: 3\n,"1. G. Avelino, E. Constantinou, M. T. Valente ..."
2,ReACT-1,Ensure a team of friendly and supportive\ndeve...,c_edges,The number of active edges in the technical ne...,No. of reference: 3\n,"1. G. Avelino, E. Constantinou, M. T. Valente ..."
3,ReACT-1,Ensure a team of friendly and supportive\ndeve...,c_long_tail,The degree of the 75𝑡ℎ percentile of nodes in ...,No. of reference: 3\n,"1. G. Avelino, E. Constantinou, M. T. Valente ..."
4,ReACT-1,Ensure a team of friendly and supportive\ndeve...,c_mean_degree,sum of all nodes’ degree divided by the number...,No. of reference: 3\n,"1. G. Avelino, E. Constantinou, M. T. Valente ..."
...,...,...,...,...,...,...
345,ReACT-98,Make it easy for newcomers build the system\nl...,c_mean_degree,sum of all nodes’ degree divided by the number...,No. of reference: 1,"1. Steinmacher I, Gerosa MA. Fostering Free/Li..."
346,ReACT-99,Keep the issue list clean and triaged: Read th...,c_nodes,The number of active nodes in the technical ne...,No. of reference: 2,"1. Steinmacher I, Gerosa MA. Fostering Free/Li..."
347,ReACT-99,Keep the issue list clean and triaged: Read th...,c_edges,The number of active edges in the technical ne...,No. of reference: 2,"1. Steinmacher I, Gerosa MA. Fostering Free/Li..."
348,ReACT-99,Keep the issue list clean and triaged: Read th...,c_long_tail,The degree of the 75𝑡ℎ percentile of nodes in ...,No. of reference: 2,"1. Steinmacher I, Gerosa MA. Fostering Free/Li..."


## IG Graph, Pie Plot and Scattered Plot - ALL

In [19]:
df = ReAct_Title_Feature_FeatureDescription_Reference
ReACT_number = df['ReACT_number'].values
ReACT_Title = df['ReACT_title'].values
Feature = df['Features'].values
df['FeatureDescription'] = df['FeatureDescription'].apply(wrap_text)
FeatureDescription = df['FeatureDescription'].values

Edges = ReAct_Title_Feature_FeatureDescription_Reference['References'].values+'\n Reference(s) \n'+ReAct_Title_Feature_FeatureDescription_Reference['Articles'].apply(wrap_text)



net = Network(directed =True)
for i in range(0,len(ReACT_number)):
 # print(actionable[i])
 net.add_node(ReACT_number[i], label=ReACT_number[i], title=ReACT_Title[i])
 net.add_node(Feature[i], label=Feature[i], title=FeatureDescription[i], color = "#E07772",shape='box')
 net.add_edge(ReACT_number[i], Feature[i], weight=extract_numbers(Edges[i]), title=Edges[i], color = "#898980")

#  net.add_edge(ReACT_number[i], Feature[i], weight=extract_numbers(Edges[i]), title=Edges[i], color = "#898980")

# net.show_buttons(filter_=['physics'])
net.show('IG_Feature.html')

In [20]:
ReAct_Title_Feature_FeatureDescription_Reference['Features'].value_counts()

Features
c_nodes           48
num_act_dev       44
c_edges           42
c_long_tail       42
c_mean_degree     41
num_commits       24
top_c_fract       18
e_nodes           17
e_edges           13
e_mean_degree     12
e_long_tail       12
c_c_coef          12
num_files          8
top_e_fract        7
num_emails         7
e_interruption     3
Name: count, dtype: int64

### Need to combine the following dataframe with the Articles, for TOOL Dynamicity (Searching for ReACTs based on publication profile, years, etc)

In [21]:
ReAct_Title_Feature_FeatureDescription_Reference

,ReACT_number,ReACT_title,Features,FeatureDescription,References,Articles
0,ReACT-1,Ensure a team of friendly and supportive\ndeve...,num_act_dev,The number of Active Developers is the count o...,No. of reference: 3\n,"1. G. Avelino, E. Constantinou, M. T. Valente ..."
1,ReACT-1,Ensure a team of friendly and supportive\ndeve...,c_nodes,The number of active nodes in the technical\nn...,No. of reference: 3\n,"1. G. Avelino, E. Constantinou, M. T. Valente ..."
2,ReACT-1,Ensure a team of friendly and supportive\ndeve...,c_edges,The number of active edges in the technical\nn...,No. of reference: 3\n,"1. G. Avelino, E. Constantinou, M. T. Valente ..."
3,ReACT-1,Ensure a team of friendly and supportive\ndeve...,c_long_tail,The degree of the 75𝑡ℎ percentile of nodes in ...,No. of reference: 3\n,"1. G. Avelino, E. Constantinou, M. T. Valente ..."
4,ReACT-1,Ensure a team of friendly and supportive\ndeve...,c_mean_degree,sum of all nodes’ degree divided by the number...,No. of reference: 3\n,"1. G. Avelino, E. Constantinou, M. T. Valente ..."
...,...,...,...,...,...,...
345,ReACT-98,Make it easy for newcomers build the system\nl...,c_mean_degree,sum of all nodes’ degree divided by the number...,No. of reference: 1,"1. Steinmacher I, Gerosa MA. Fostering Free/Li..."
346,ReACT-99,Keep the issue list clean and triaged: Read th...,c_nodes,The number of active nodes in the technical\nn...,No. of reference: 2,"1. Steinmacher I, Gerosa MA. Fostering Free/Li..."
347,ReACT-99,Keep the issue list clean and triaged: Read th...,c_edges,The number of active edges in the technical\nn...,No. of reference: 2,"1. Steinmacher I, Gerosa MA. Fostering Free/Li..."
348,ReACT-99,Keep the issue list clean and triaged: Read th...,c_long_tail,The degree of the 75𝑡ℎ percentile of nodes in ...,No. of reference: 2,"1. Steinmacher I, Gerosa MA. Fostering Free/Li..."


### Pie Plot - ALL

In [22]:
unique_counts = ReAct_Title_Feature_FeatureDescription_Reference['Features'].value_counts().reset_index()

# Rename the columns for clarity
unique_counts.columns = ['Feature Name', 'Count']

# Define an array of colors
darker_shades = ['darkred', 'lightblue', 'lightgreen', '#FFABAB', '#91a8a8',
                 '#95ab8a', '#00a86b', '#8E44AD', '#2E4053', '#D35400', '#a37b65',
                 '#6C5B7B', '#C0C999', '#7D8CC4', '#FFA07A', '#88D8B0',
    '#826c7F', '#D8D8D8', '#5F758E', '#E2B464', '#649E7D',
    '#CAAE77', '#7A9E9F', '#FAE5D3', '#BAC9FF', '#E47878',
    '#D2F2D2', '#7C83FD', '#F5A623', '#A77B7B', '#D8D8D8']


# Create an interactive bar plot using Plotly Express
fig = px.pie(unique_counts, names='Feature Name', values='Count', color='Feature Name', color_discrete_sequence=darker_shades)

# Save the plot as an HTML file
fig.write_html('PiePlot_Feature.html')
# fig.show()

### SP - ALL

In [24]:
import plotly.graph_objects as go
import plotly.io as pio
subject = ReAct_Title_Feature_FeatureDescription_Reference['Features']
score = ReAct_Title_Feature_FeatureDescription_Reference['ReACT_number']

TextToShow = ReAct_Title_Feature_FeatureDescription_Reference['ReACT_number'] + " Positively Impacts "
TextToShow += ReAct_Title_Feature_FeatureDescription_Reference['Features'] + "<br>"
TextToShow +=  "<b>"+ReAct_Title_Feature_FeatureDescription_Reference['ReACT_number']+"</b>: "
TextToShow += ReAct_Title_Feature_FeatureDescription_Reference['ReACT_title'].apply(wrap_text_withBR)
TextToShow += "<br><b>" +ReAct_Title_Feature_FeatureDescription_Reference['Features'] +": </b>"
TextToShow += ReAct_Title_Feature_FeatureDescription_Reference['FeatureDescription'].apply(wrap_text_withBR)

data = [dict(
  type = 'scatter',
  x = subject,
  y = score,
  mode = 'markers',
  text = TextToShow,
  transforms = [dict(
    type = 'groupby',
    groups = subject
  )],
   marker=dict(
        symbol='square',  # Set the shape of the marker (e.g., 'circle', 'square', 'diamond', etc.)
        size=10  # Set the size of the circles (adjust as needed)

    )
)]

layout = dict(xaxis=dict(title='Features'),
              yaxis=dict(title='ReACT Number'),
              height=1200,
              width=800)

fig_dict = dict(data=data, layout=layout)
# pio.show(fig_dict, validate=False)
pio.write_html(fig_dict, file='SP_Feature.html', auto_open=True, validate=False)